In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
from sys import exit
from matplotlib.backends.backend_pdf import PdfPages 
import matplotlib.dates as mdates
import seaborn as sns
from calendar import monthrange
import datetime
from tqdm import tqdm
from tabulate import tabulate
import warnings
warnings.filterwarnings("ignore")

In [2]:
def Create_PDFs(Filename,Identifier): 
    p = PdfPages(Filename) 
    fig_nums = plt.get_fignums()   
    figs = [plt.figure(n) for n in fig_nums] 
    #for fig in tqdm(figs, desc=f'  Plotting {Identifier} ...'):
    for fig in figs:  
        fig.savefig(p, format='pdf')  
    p.close()
#
def GetCoeffVariation (listofval): # Co-efficient of variation (CV)
    MeanVal = sum(listofval)/len(listofval)
    StdDev = np.std(listofval)
    # Standard deviation divided by the mean, lower the better (more stable).
    VolRatio = abs(StdDev/MeanVal) 
    return (round(VolRatio,5))
#
def ApplySunHours(Year,Month):
    SunHours = SunHoursDF.loc[(SunHoursDF["Year"] == Year) & (SunHoursDF["Month"] == Month)]["SunHours"].values[0]
    return SunHours
#
def Average(numberslist):
    return (round(sum(numberslist)/len(numberslist),2))
def MaxValue(numberslist):
    return (max(numberslist))
def MinValue(numberslist):
    return (min(numberslist))

In [3]:
Profiles = {
    'Profile_A':[18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18],
    'Profile_B':[18,18,18,18,18,18,20,20,20,18,18,18,18,18,18,18,20,20,20,20,20,20,18,18],
    'Profile_C':[20,20,20,20,20,20,18,18,18,20,20,20,20,20,20,20,18,18,18,18,18,18,20,20],
    'Profile_D':[18,18,18,18,18,18,21,21,21,18,18,18,18,18,18,18,21,21,21,21,21,21,18,18],
}
NoOfHP = 30 # in million

In [4]:
def ResidentialHP_A0(Profile,OutT,Hour,Month,UK_HTC): # 'ResidentialHP_A' HP always operating during Winter and Summer.
    HeatLossThermal = UK_HTC*126.57*1.25 # 1.8W/m2K times 126.57m2*1.25 to include floors and roofs.
    InsideTemp = Profiles[Profile][Hour]
    dT = abs(InsideTemp-OutT)
    COP = getCOP(dT,InsideTemp,Month)
    HeatPumpElec = round((HeatLossThermal*dT/COP)*NoOfHP,2) # in MWe
    #print(f"A: Profile: {Profile} InsideTemp:{InsideTemp} OutT:{OutT} dT:{dT} COP:{COP} Hour:{Hour} Month:{Month} UK_HTC:{UK_HTC} HeatPumpElec:{HeatPumpElec}.")
    return(HeatPumpElec)
#
InHighTemp = 25
def ResidentialHP_B0(Profile,OutT,Hour,Month,UK_HTC,AirToAirAdoptionRate): # 'ResidentialHP_B' HP always operating during winter, during summer operates till IAT reaches 25C; In Summer less than 25 the HP power use is Zero.
    HeatLossThermal = UK_HTC*126.57*1.25 # 1.8W/m2K times 126.57m2*1.25 to include floors and roofs.
    InsideTemp = Profiles[Profile][Hour]
    if (InsideTemp-OutT) > 0:                               # Winter time, Heating required.
        dT = InsideTemp-OutT
        COP = getCOP(dT,InsideTemp,Month)
        HeatPumpElec = round((HeatLossThermal*dT/COP)*NoOfHP,2) # in MWe
        #print(f"B1: Profile: {Profile} InsideTemp:{InsideTemp} OutT:{OutT} dT:{dT} COP:{COP} Hour:{Hour} Month:{Month} UK_HTC:{UK_HTC} AirToAirAdoptionRate:{AirToAirAdoptionRate} HeatPumpElec:{HeatPumpElec}.")
        return HeatPumpElec
    elif (InsideTemp-OutT) < 0 and (OutT >= InHighTemp): # Summer, Cooling till 25C required - Hot summer
        dT = OutT-InHighTemp
        COP = getCOP(dT,InsideTemp,Month)
        HeatPumpElec = round((HeatLossThermal*dT/COP)*NoOfHP*AirToAirAdoptionRate/100,2) # in MWe
        #print(f"B2: Profile: {Profile} InsideTemp:{InsideTemp} OutT:{OutT} dT:{dT} COP:{COP} Hour:{Hour} Month:{Month} UK_HTC:{UK_HTC} AirToAirAdoptionRate:{AirToAirAdoptionRate} HeatPumpElec:{HeatPumpElec}.")
        return HeatPumpElec
    else:                                                   # Warm but but not hot.
        #print(f"B3: Profile: {Profile} InsideTemp:{InsideTemp} OutT:{OutT} Hour:{Hour} Month:{Month} UK_HTC:{UK_HTC} HeatPumpElec: Zero.")
        return (float(0))

In [5]:
def ResidentialHP_A(OutTemp,Month,UK_HTC): # 'ResidentialHP_A' HP always operating during Winter and Summer.
    if len(OutTemp) != 24:
        exit(f"24 number of outside temp not provided. Actual {len(OutTemp)}. Exiting...")
    HeatLossThermal = UK_HTC*126.57*1.25 # 1.8W/m2K times 126.57m2*1.25 to include floors and roofs.
    HPPowerUse_E = []
    for OutT,InsideTemp in zip(OutTemp,CoolingProfile):
        dT = abs(InsideTemp-OutT)
        COP = getCOP(dT,InsideTemp,Month)
        HeatPumpElec = round((HeatLossThermal*dT/COP)*NoOfHP,2) # in MWe
        HPPowerUse_E.append(HeatPumpElec)
    return(HPPowerUse_E)
#
InHighTemp = 25
def ResidentialHP_B(OutTemp,Month,AirToAirAdoptionRate,UK_HTC): 
    # 'ResidentialHP_B' HP always operating during winter, during summer operates till IAT reaches 25C; In Summer less than 25 the HP power use is Zero.
    if len(OutTemp) != 24:
        exit(f"24 number of outside temp not provided. Actual {len(OutTemp)}. Exiting...")
    HeatLossThermal = UK_HTC*126.57*1.25 # 1.8W/m2K times 126.57m2*1.25 to include floors and roofs.
    HPPowerUse_E = []
    for OutT,InsideTemp in zip(OutTemp,CoolingProfile):
        if (InsideTemp-OutT) > 0:                               # Winter time, Heating required.
            dT = InsideTemp-OutT
            COP = getCOP(dT,InsideTemp,Month)
            HeatPumpElec = round((HeatLossThermal*dT/COP)*NoOfHP,2) # in MWe
            HPPowerUse_E.append(HeatPumpElec)
        elif (InsideTemp-OutT) < 0 and (OutT >= InHighTemp): # Summer, Cooling till 25C required - Hot summer
            dT = OutT-InHighTemp
            COP = getCOP(dT,InsideTemp,Month)
            HeatPumpElec = round((HeatLossThermal*dT/COP)*NoOfHP*AirToAirAdoptionRate/100,2) # in MWe
            HPPowerUse_E.append(HeatPumpElec)
        else:                                                   # Warm but but not hot.
            HPPowerUse_E.append(float(0))
    return(HPPowerUse_E)

In [6]:
def getCOP(dT,InsideTemp,Month):
    SoilTemp = {1:6.079032,2:5.880090,3:6.188341,4:8.265506,5:11.297607,6:14.157917, # Average Soil temperature in Shefffield
                7:15.928495,8:16.968100,9:15.952500,10:13.576498,11:11.006667,12:8.050320}
    #
    GSHPdT = abs(InsideTemp-SoilTemp[Month])
    WSHPdT = abs(10-InsideTemp)
    #
    #ASHPCOP = 6.08-0.09*dT+0.0005*dT*dT       # As Literature.
    ASHPCOP = 4.08-0.09*dT+0.0008*dT*dT        # As evaluated from the Heat Pump demonstration project.
    GSHPCOP = 10.29-0.21*GSHPdT+0.0012*GSHPdT*GSHPdT  # As Literature.
    WSHPCOP = 9.97-0.20*WSHPdT+0.0012*WSHPdT*WSHPdT   # As Literature.
    # Weighted Average, Roughly even distribution of ASHP, and GSHP and WSHP (district heating).
    WAverage = (0.5*ASHPCOP)+(0.3*GSHPCOP)+(0.2*WSHPCOP)
    return (WAverage)

In [7]:
Month = {1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',
         7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
Seasons = {1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',6:'Summer',
         7:'Summer',8:'Summer',9:'Autumn',10:'Autumn',11:'Autumn',12:'Winter'}
DayName = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
WeekDayWeekend = {0:'Weekday',1:'Weekday',2:'Weekday',3:'Weekday',4:'Weekday',5:'WeekEnd',6:'WeekEnd'}
MonthlyDaylight = {1:8.4,2:10.1,3:11.9,4:14.0,5:15.7,6:16.7,\
                   7:16.2,8:14.7,9:12.7,10:10.8,11:8.9,12:7.9} # SOURCE: https://www.worlddata.info/europe/united-kingdom/sunset.php

In [8]:
# For Hourly
HourlyDF = pd.read_csv("0.DataSource/National Demand/df_fuel_ckan_DemandPriceTemp_Hourly.csv")
HourlyDF['DATETIME'] = pd.to_datetime(HourlyDF['DATETIME'])
HourlyDF = HourlyDF.loc[(HourlyDF['DATETIME'].dt.year >= 2009) & (HourlyDF['DATETIME'].dt.year <= 2023)]
#
HourlyDF['Year'],HourlyDF['Month'],HourlyDF['Day'],HourlyDF['WeekNumber'],HourlyDF['DaysOfTheYear'],\
    HourlyDF['DayOfTheWeek'],HourlyDF['Hour'] = HourlyDF['DATETIME'].dt.year,\
    HourlyDF['DATETIME'].dt.month,HourlyDF['DATETIME'].dt.day,HourlyDF['DATETIME'].dt.isocalendar().week,\
    HourlyDF['DATETIME'].dt.dayofyear,HourlyDF['DATETIME'].dt.dayofweek,HourlyDF['DATETIME'].dt.hour
HourlyDF['Seasons'] = HourlyDF['Month'].map(Seasons)
HourlyDF['WeekDay_OR_Weekend'] = HourlyDF['DayOfTheWeek'].map(WeekDayWeekend)
HourlyDF.insert(2,'Seasons',HourlyDF.pop('Seasons'))

In [9]:
SunHoursDF = pd.read_csv("0.DataSource/SheffieldDayLightHours.csv") # Monthly SunHours from Sheffield ## https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/sheffielddata.txt

In [10]:
def DataTableOption(Plan,AirToAirAdoptionRate,UK_HTC):
    DataTable = pd.DataFrame(columns=['Date','Year','Month','Day','DayOfTheYear',
                                      'Day Avg Demand','Day Max Demand','Day Avg HPe','Day Max HPe','Min Demand With HP','Avg Demand With HP','Max Demand With HP',
                                      'DayDemand_CV','DayHP_CV','DayDemandHPCombined_CV','DayDemandToHP_CV_Change',
                                      'Demand profile, MWe','HP Profile, MWe','Demand profile with HP, MWe',
                                      'Temperature Profile,C'])
    StartingYear = 2009
    EndYear = 2022
    Years = tqdm(range(StartingYear,EndYear+1))
    for Year in Years:
        ValueYEAR = HourlyDF.loc[(HourlyDF['Year'] == Year)]
        YearDay = 0
        for Month in range(1,12+1):
            ValueMONTH = ValueYEAR.loc[(ValueYEAR['Month'] == Month)]
            Days = monthrange(Year,Month)[1]
            Years.set_description(f'{Year}-{Month} ##>>')
            for Day in range(1,Days+1):
                Date = datetime.date(Year,Month,Day)
                DataList = []
                YearDay += 1
                ValueDAY = ValueMONTH.loc[(ValueMONTH['Day'] == Day)]
                #
                # Profiles and Lists
                DailyDemandList = ValueDAY['NATIONAL DEMAND'].tolist() # National Demand
                DailyTempList = ValueDAY['Air_Temperature'].tolist()
                #
                if Plan == 'A':
                    HPProfileMWe = ResidentialHP_A(DailyTempList,Month,UK_HTC)                        # Just HP
                elif Plan == 'B':
                    HPProfileMWe = ResidentialHP_B(DailyTempList,Month,AirToAirAdoptionRate,UK_HTC)   # Just HP
                else:
                    exit("Valid Option not received.")
                #
                DemandProfileWithHPMWe = np.array(DailyDemandList)+np.array(HPProfileMWe)  # National Demand + HP
                #
                # Average Values
                DayMEANDemand = Average(DailyDemandList)               # National Demand
                DayMEANHPe = Average(HPProfileMWe)                     # Just HP
                DayMEANDemandAndHPe = Average(DemandProfileWithHPMWe)  # National Demand + HP
                #
                # Min Values
                DayMINDemandAndHPe = MinValue(DemandProfileWithHPMWe)  # National Demand + HP
                #
                # Max Values
                DayMAXDemand = MaxValue(DailyDemandList)               # National Demand
                DayMAXHPe = MaxValue(HPProfileMWe)                     # Just HP
                DayMAXDemandAndHPe = MaxValue(DemandProfileWithHPMWe)  # National Demand + HP
                #
                # CV Values
                DayCV_Demand = GetCoeffVariation(DailyDemandList)               # National Demand
                DayCV_HPe = GetCoeffVariation(HPProfileMWe)                     # Just HP
                DayCV_DemandAndHPe = GetCoeffVariation(DemandProfileWithHPMWe)  # National Demand + HP 
                DayCV_Delta = DayCV_Demand - DayCV_DemandAndHPe                 # Coefficient of Varience change from Demand and (Demand + HP). +ve means it became more stable, -ve means it got worse.
                #
                DataList.extend([Date,Year,Month,Day,YearDay,DayMEANDemand,DayMAXDemand,DayMEANHPe,DayMAXHPe,DayMINDemandAndHPe,DayMEANDemandAndHPe,DayMAXDemandAndHPe])
                DataList.extend([DayCV_Demand,DayCV_HPe,DayCV_DemandAndHPe,DayCV_Delta])
                DataList.append(DailyDemandList)
                DataList.append(HPProfileMWe)
                DataList.append(DemandProfileWithHPMWe.tolist())
                DataList.append(DailyTempList)
                #
                DataTable.loc[len(DataTable), DataTable.columns] = (DataList)
    DataTable['Seasons'] = DataTable['Month'].map(Seasons)
    DataTable.insert(3,'Seasons',DataTable.pop('Seasons'))
    return DataTable

In [11]:
def DataTableOptionTesting(Profile,Plan,AirToAirAdoptionRate,UK_HTC):
    DataTable = pd.DataFrame(columns=['Date','Hour','Current Demand','HP Demand','Combined Demand'])
    StartingYear = 2009
    EndYear = 2022
    Years = tqdm(range(StartingYear,EndYear+1))
    for Year in Years:
        ValueYEAR = HourlyDF.loc[(HourlyDF['Year'] == Year)]
        YearDay = 0
        for Month in range(1,12+1):
            ValueMONTH = ValueYEAR.loc[(ValueYEAR['Month'] == Month)]
            Days = monthrange(Year,Month)[1]
            for Day in range(1,Days+1):
                Years.set_description(f'{Year}-{Month}-{Day} ##>>')
                Date = datetime.date(Year,Month,Day)
                ValueDAY = ValueMONTH.loc[(ValueMONTH['Day'] == Day)]
                for Hour in range (0,24):
                    DataList = []
                    ValueHOUR = ValueDAY.loc[(ValueDAY['Hour'] == Hour)]
                    #
                    # Profiles and Lists
                    Demand = ValueHOUR['NATIONAL DEMAND'].values[0] # National Demand
                    ExtTemp = ValueHOUR['Air_Temperature'].values[0]
                    #
                    if Plan == 'A':
                        HPDemand = ResidentialHP_A0(Profile,ExtTemp,Hour,Month,UK_HTC)                        # Just HP
                    elif Plan == 'B':
                        HPDemand = ResidentialHP_B0(Profile,ExtTemp,Hour,Month,UK_HTC,AirToAirAdoptionRate)   # Just HP
                    else:
                        exit("Valid Option not received.")
                    TotalDemand = Demand + HPDemand
                    DataList.extend([Date,Hour,Demand,HPDemand,TotalDemand])
                    DataTable.loc[len(DataTable), DataTable.columns] = (DataList)
    return DataTable

In [87]:
Profile = 'Profile_A'
Plan = 'B'
AirToAirAdoptionRate = 60
UK_HTC = 1.8
ReturnValues = DataTableOptionTesting(Profile,Plan,AirToAirAdoptionRate,UK_HTC)
ReturnValues.to_csv("DemandWithHP_1.8_ProfileA_PlanB_A2AAdoptRate_60.csv")

2022-12 ##>>: 100%|██████████| 14/14 [11:55<00:00, 51.11s/it]


In [12]:
Profile = 'Profile_A'
Plan = 'B'
AirToAirAdoptionRate = 0
UK_HTC = 1.8
ReturnValues = DataTableOptionTesting(Profile,Plan,AirToAirAdoptionRate,UK_HTC)
ReturnValues.to_csv("./0.DataSource/National Demand/DemandWithHP_1.8_ProfileA_PlanB_A2AAdoptRate_0.csv")

2022-12-31 ##>>: 100%|██████████| 14/14 [13:22<00:00, 57.30s/it]


In [13]:
Profile = 'Profile_A'
Plan = 'B'
AirToAirAdoptionRate = 100
UK_HTC = 1.8
ReturnValues = DataTableOptionTesting(Profile,Plan,AirToAirAdoptionRate,UK_HTC)
ReturnValues.to_csv("./0.DataSource/National Demand/DemandWithHP_1.8_ProfileA_PlanB_A2AAdoptRate_100.csv")

2022-12-31 ##>>: 100%|██████████| 14/14 [13:04<00:00, 56.02s/it]


In [85]:
Profile = 'Profile_A'
Plan = 'B'
AirToAirAdoptionRate = 60
UK_HTC = 1.8
CoolingProfile = Profiles[Profile]
DataTableValues = DataTableOption(Plan,AirToAirAdoptionRate,UK_HTC)

2022-12 ##>>: 100%|██████████| 14/14 [00:18<00:00,  1.30s/it]


In [86]:
DataTableValues['Demand profile with HP, MWe'][0]

0       [69935.43, 68859.93, 66625.28, 64601.380000000...
1       [60519.58, 59185.68, 58330.72, 57232.229999999...
2       [61758.729999999996, 60585.22, 60181.15, 59325...
3       [63180.630000000005, 62173.9, 61698.07, 60288....
4       [59756.7, 60368.369999999995, 60071.3699999999...
                              ...                        
5108    [43550.0, 44662.69, 43750.18, 43257.19, 42851....
5109    [35241.53, 35210.09, 35001.5, 36055.6, 36602.7...
5110    [32459.47, 32989.83, 32473.34, 36550.33, 34634...
5111    [44596.14, 43116.0, 39386.89, 36771.41, 34547....
5112    [35944.22, 35583.72, 34494.72, 33472.34, 33150...
Name: Demand profile with HP, MWe, Length: 5113, dtype: object

In [14]:
Values = ['WIND','SOLAR','EMBEDDED_WIND_GENERATION','EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION','EMBEDDED_SOLAR_CAPACITY']
PvtTable = pd.pivot_table(HourlyDF, values=Values, index='Year',aggfunc='max')
PvtTable/1000

,EMBEDDED_SOLAR_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_WIND_GENERATION,SOLAR,WIND
Year,,,,,,
2009,0.000,0.000,1.773,1.6915,0.0000,1.5685
2010,0.079,0.015,2.236,2.0720,0.0000,2.5785
2011,1.121,0.154,1.846,2.0885,0.0000,4.1870
2012,2.035,0.953,2.085,1.8030,0.0000,5.8100
2013,3.345,2.715,2.486,1.8820,1.7440,6.8090
2014,5.991,3.815,4.315,2.9100,3.0240,8.9235
2015,9.063,6.995,4.163,3.6770,5.9530,8.9130
2016,11.503,7.390,5.113,4.1120,8.2660,10.4780
2017,12.916,8.880,5.803,4.5325,9.0565,12.3205


In [13]:
HourlyDF.columns

Index(['DATETIME', 'MC_StartingVolume', 'Seasons', 'SBP(GBP/MWh)',
       'SSP(GBP/MWh)', 'MID_APX_VOLUME(MWh)', 'MID_APX_PRICE(GBP/MWh)',
       'MID_N2EX_VOLUME(MWh)', 'MID_N2EX_PRICE(GBP/MWh)', 'SSP_(GBP/MWh)',
       'NIV(MWh)', 'NordPool_HH_Vol(MWh)', 'NordPool_HH_Price(GBP/MWh)',
       'NordPool_DA_H_Vol(MWh)', 'NordPool_DA_H_Price(GBP/MWh)',
       'Air_Temperature', 'GAS', 'COAL', 'NUCLEAR', 'WIND', 'HYDRO', 'IMPORTS',
       'BIOMASS', 'OTHER', 'SOLAR', 'STORAGE', 'GENERATION',
       'CARBON_INTENSITY', 'LOW_CARBON', 'ZERO_CARBON', 'RENEWABLE', 'FOSSIL',
       'GAS_perc', 'COAL_perc', 'NUCLEAR_perc', 'WIND_perc', 'HYDRO_perc',
       'IMPORTS_perc', 'BIOMASS_perc', 'OTHER_perc', 'SOLAR_perc',
       'STORAGE_perc', 'GENERATION_perc', 'LOW_CARBON_perc',
       'ZERO_CARBON_perc', 'RENEWABLE_perc', 'FOSSIL_perc',
       'ENGLAND_WALES_DEMAND', 'TRANSMISSION_SYSTEM_DEMAND', 'NATIONAL DEMAND',
       'ProfileAPlanB_A2A_0Per_HP_Only_Demand',
       'ProfileAPlanB_A2A_0Per_COMBIN